# Setup

In [1]:
import pandas as pd
import numpy as numpy
import os
import json
from pathlib import Path
from tqdm.notebook import tqdm
from datetime import datetime

# Own functions
%load_ext autoreload
%autoreload 2

from utils import query, ETL
from utils.helpers import *

## Load api key

In [2]:
keys_path = "keys/api_key.json"
# Check if existing keys
if not os.path.exists(keys_path):
    keys = {}
    while True:
        # In case it does not exist, the following are requested 
        name_key = str(input("Enter name of your API key: "))
        api_key = str(input("Enter your API key token: "))

        keys[name_key] = api_key

        answer = str(input("Other API key? y/n")).lower()

        if answer in ["y", "yes", "n", "no"]:
            Path('keys').mkdir(parents=True, exist_ok=True)
            if answer in ["n", "no"]:
                # Finally save keys
                with open(keys_path, 'w') as file:
                    json.dump(keys, file)
                break
        else:
            while answer not in ["y", "yes", "n", "no"]:
                answer = str(input("Invalid answer\n. Other API key? y/n")).lower()


    
with open(keys_path, 'r') as file:
    keys_data = json.load(file) 

printg("API keys loaded successfully")             

API keys loaded successfully


# Load data

In [3]:

data_path = "data/data.csv"
# Check if existing data
if not os.path.exists(data_path):
    Path('data').mkdir(parents=True, exist_ok=True)
    printy("Start search")
    # Extract first page
    dict_data_result = query.searchAPI(keys_data["hubspot"])
    # Convert in DataFrame
    data = [ETL.dict2dataframe(dict_data_result)]
    
    data_size = dict_data_result["total"]
    iterations = data_size // 100 if data_size % 100 == 0 else (data_size // 100) + 1
    limit = 100
    for i in tqdm(range(1,iterations), desc="Extracting"):
        if data_size - (i*100) <= 100:
            limit = data_size - (i*100)
        dict_data_result = query.searchAPI(keys_data["hubspot"], limit = limit, page = i)
        # Convert in DataFrame
        data.extend([ETL.dict2dataframe(dict_data_result)])
    printg("Finish search")
    # Save
    data = pd.concat(data)
    data.to_csv(data_path, sep = "|", index = False)

else:
    print("Exist data.csv")
    printy("Start load")
    data = pd.read_csv(data_path, sep = "|")
    printg("Finish load")


print("\t Total elements", len(data))

Exist data.csv
Start load
Finish load
	 Total elements 6936


In [4]:
data = data.reset_index(drop= True)
data.tail()

,address,country,createdate,firstname,hs_object_id,industry,lastmodifieddate,lastname,phone,raw_email,technical_test___create_date
6931,"Spruce Grove, 6997",Milton Keynes,2023-05-15T02:39:02.017Z,Violet,413209,Fruit and vegetables,2023-09-16T10:58:09.029Z,Moran,0-804-316-088,Violet <violet_moran1613873168@gembat.biz> Con...,2021-02-11
6932,"King William Rue, 748",Cork,2023-05-15T02:39:02.017Z,Vera,413901,Dairy products,2023-09-16T10:55:10.641Z,Payne,3-668-267-421,Vera <vera_payne1587262285@gmail.com> Contact ...,2021-01-10
6933,"Udall Street, 7289",London,2023-05-15T02:39:02.017Z,Vera,421501,Milling,2023-09-16T10:53:10.027Z,Payne,4-735-376-611,Vera <vera_payne1227725435@irrepsy.com> Contac...,2022-01-01
6934,"Belmont Park Walk, 5338",Dublin,2023-05-15T02:39:02.017Z,Vera,421753,Poultry and fish,2023-09-16T10:55:40.011Z,Janes,5-655-825-605,Vera <vera_janes1928590357@nanoff.biz> Contact...,2021-11-18
6935,"Woodland Hill, 8430",Milton Keynes,2023-05-15T02:39:02.017Z,Vicky,422853,Dairy products,2023-09-16T10:53:28.891Z,Morrison,7-777-162-004,Vicky <vicky_morrison114799425@atink.com> Cont...,2021-07-16


# Transform data

## Simple transformation

In [5]:
data["createdate"] = data["createdate"].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))

## Search Country

In [6]:
set(list(data["country"].values))

{'Cork',
 'Dublin',
 'England',
 'Ireland',
 'Limerick',
 'London',
 'Milton Keynes',
 'Oxford',
 'Plymouth',
 'Waterford',
 'Winchester',
 nan}

In [7]:
data[["contry_found","city_found"]] = data["country"].apply(lambda x: pd.Series(ETL.found_contry(x)))

SubdivisionHierarchy(code='IE-WD', country_code='IE', name='Waterford', parent='M', parent_code='IE-M', type='County')
SubdivisionHierarchy(code='IE-D', country_code='IE', name='Dublin', parent='L', parent_code='IE-L', type='County')
SubdivisionHierarchy(code='IE-D', country_code='IE', name='Dublin', parent='L', parent_code='IE-L', type='County')
SubdivisionHierarchy(code='IE-LK', country_code='IE', name='Limerick', parent='M', parent_code='IE-M', type='County')
SubdivisionHierarchy(code='IE-D', country_code='IE', name='Dublin', parent='L', parent_code='IE-L', type='County')
SubdivisionHierarchy(code='IE-LK', country_code='IE', name='Limerick', parent='M', parent_code='IE-M', type='County')
winchester
SubdivisionHierarchy(code='GB-MIK', country_code='GB', name='Milton Keynes', parent='GB-ENG', parent_code='GB-GB-ENG', type='Unitary authority')
SubdivisionHierarchy(code='IE-WD', country_code='IE', name='Waterford', parent='M', parent_code='IE-M', type='County')
SubdivisionHierarchy(code

In [8]:
# Errors
data[data["contry_found"] == "error"][["country","contry_found","city_found"]].tail()

,country,contry_found,city_found
6892,Winchester,error,Winchester
6901,Winchester,error,Winchester
6920,Winchester,error,Winchester
6929,Winchester,error,Winchester
6930,Winchester,error,Winchester


In [9]:
# Same error?
data[data["contry_found"] == "error"]["country"].unique()

array(['Winchester', nan], dtype=object)

In [10]:
# review Nan data
data[~data["country"].isna()].head()

,address,country,createdate,firstname,hs_object_id,industry,lastmodifieddate,lastname,phone,raw_email,technical_test___create_date,contry_found,city_found
0,"Blackpool Rue, 6576",Waterford,2023-05-15 02:39:02.002,Zoe,416102,Poultry and fish,2023-09-16T10:56:44.913Z,Owen,0-774-386-624,Zoe <zoe_owen450104633@acrit.org> Contact Info.,2021-07-13,Ireland,Waterford
1,"Parkfield Avenue, 5340",Ireland,2023-05-15 02:39:02.003,Zara,413403,Fruit and vegetables,2023-09-16T10:55:08.803Z,Rodwell,6-777-367-783,Zara <zara_rodwell1398442854@nickia.com> Conta...,2021-01-09,Ireland,Ireland
2,"Abourne Lane, 876",Ireland,2023-05-15 02:39:02.003,Zara,417951,Milling,2023-09-16T10:53:14.079Z,Freeburn,5-618-556-540,Zara <zara_freeburn1593147546@gmail.com> Conta...,2021-08-30,Ireland,Ireland
3,"Chester Crossroad, 7070",Dublin,2023-05-15 02:39:02.003,Winnie,419852,Dairy products,2023-09-16T10:55:08.759Z,Walter,1-161-604-327,Winnie <winnie_walter538064895@sheye.org> Cont...,2021-02-10,Ireland,Dublin
4,"Tilloch Crossroad, 8332",Dublin,2023-05-15 02:39:02.003,Zoe,425352,Meat,2023-09-16T10:55:09.707Z,Owen,5-645-416-200,Zoe <zoe_owen1652446013@bungar.biz> Contact Info.,2021-11-02,Ireland,Dublin


In [11]:
# Both the country and number information are empty, and it is evident that the records are repeated, so they will be eliminated
data = data[~data["country"].isna()]
data = data.reset_index(drop= True)
data.head()

,address,country,createdate,firstname,hs_object_id,industry,lastmodifieddate,lastname,phone,raw_email,technical_test___create_date,contry_found,city_found
0,"Blackpool Rue, 6576",Waterford,2023-05-15 02:39:02.002,Zoe,416102,Poultry and fish,2023-09-16T10:56:44.913Z,Owen,0-774-386-624,Zoe <zoe_owen450104633@acrit.org> Contact Info.,2021-07-13,Ireland,Waterford
1,"Parkfield Avenue, 5340",Ireland,2023-05-15 02:39:02.003,Zara,413403,Fruit and vegetables,2023-09-16T10:55:08.803Z,Rodwell,6-777-367-783,Zara <zara_rodwell1398442854@nickia.com> Conta...,2021-01-09,Ireland,Ireland
2,"Abourne Lane, 876",Ireland,2023-05-15 02:39:02.003,Zara,417951,Milling,2023-09-16T10:53:14.079Z,Freeburn,5-618-556-540,Zara <zara_freeburn1593147546@gmail.com> Conta...,2021-08-30,Ireland,Ireland
3,"Chester Crossroad, 7070",Dublin,2023-05-15 02:39:02.003,Winnie,419852,Dairy products,2023-09-16T10:55:08.759Z,Walter,1-161-604-327,Winnie <winnie_walter538064895@sheye.org> Cont...,2021-02-10,Ireland,Dublin
4,"Tilloch Crossroad, 8332",Dublin,2023-05-15 02:39:02.003,Zoe,425352,Meat,2023-09-16T10:55:09.707Z,Owen,5-645-416-200,Zoe <zoe_owen1652446013@bungar.biz> Contact Info.,2021-11-02,Ireland,Dublin


In [12]:
# Set Winchester 
for i in range(len(data)):
    if data.iloc[i]["country"] == "Winchester":
        data.loc.__setitem__((i, ('contry_found')), "United Kingdom")
        data.loc.__setitem__((i, ('city_found')), "Winchester")

data[data["country"] == "Winchester"]

,address,country,createdate,firstname,hs_object_id,industry,lastmodifieddate,lastname,phone,raw_email,technical_test___create_date,contry_found,city_found
9,"Bellenden Tunnel, 9570",Winchester,2023-05-15 02:39:02.004,Willow,413251,Fruit and vegetables,2023-09-16T10:56:49.173Z,Morris,4-660-745-823,Willow <willow_morris1105172480@qater.org> Con...,2021-05-20,United Kingdom,Winchester
18,"Clerkenwell Tunnel, 4196",Winchester,2023-05-15 02:39:02.005,William,418851,Poultry and fish,2023-09-16T10:57:50.308Z,Wilkinson,8-404-135-264,William <william_wilkinson2025879958@nanoff.bi...,2021-12-07,United Kingdom,Winchester
23,"Rosewood Pass, 2189",Winchester,2023-05-15 02:39:02.005,William,434151,Milling,2023-09-16T10:58:03.394Z,Webster,6-547-630-071,William <william_webster1737049619@fuliss.net>...,2021-05-26,United Kingdom,Winchester
34,"Abbotswell Vale, 4157",Winchester,2023-05-15 02:39:02.006,William,435301,Meat,2023-09-16T10:58:03.312Z,Sanchez,5-254-677-437,William <william_sanchez384394044@jiman.org> C...,2021-03-01,United Kingdom,Winchester
58,"Baylis Hill, 7589",Winchester,2023-05-15 02:39:02.009,Wade,423601,Fruit and vegetables,2023-09-16T10:54:58.287Z,Wilton,4-584-673-556,Wade <wade_wilton1347817735@deavo.com> Contact...,2021-10-03,United Kingdom,Winchester
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6867,"Boadicea Tunnel, 3652",Winchester,2023-05-15 02:39:02.023,Tyson,422201,Bakery products,2023-09-16T10:53:26.603Z,Hall,0-411-016-277,Tyson <tyson_hall1242252066@vetan.org> Contact...,2021-05-14,United Kingdom,Winchester
6876,"Cave Alley, 8023",Winchester,2023-05-15 02:39:02.010,Wade,427701,Dairy products,2023-09-16T10:57:52.669Z,Robinson,2-488-743-044,Wade <wade_robinson2075584090@joiniaa.com> Con...,2021-05-09,United Kingdom,Winchester
6895,"Callcott Crossroad, 4279",Winchester,2023-05-15 02:39:02.012,Wade,423302,Bakery products,2023-09-16T10:57:26.388Z,Anderson,0-416-603-871,Wade <wade_anderson1689656165@cispeto.com> Con...,2021-01-28,United Kingdom,Winchester
6904,"Gathorne Street, 9970",Winchester,2023-05-15 02:39:02.013,Wade,427453,Bakery products,2023-09-16T10:55:48.223Z,Alexander,7-276-180-387,Wade <wade_alexander330181759@cispeto.com> Con...,2021-09-09,United Kingdom,Winchester


## Regrex email

In [13]:
data["email"] = data["raw_email"].apply(lambda x: ETL.find_email(x))

In [14]:
# Errors
data[data["email"] == "error"][["raw_email","email"]].tail()

,raw_email,email


In [15]:
data.head(5)

,address,country,createdate,firstname,hs_object_id,industry,lastmodifieddate,lastname,phone,raw_email,technical_test___create_date,contry_found,city_found,email
0,"Blackpool Rue, 6576",Waterford,2023-05-15 02:39:02.002,Zoe,416102,Poultry and fish,2023-09-16T10:56:44.913Z,Owen,0-774-386-624,Zoe <zoe_owen450104633@acrit.org> Contact Info.,2021-07-13,Ireland,Waterford,zoe_owen450104633@acrit.org
1,"Parkfield Avenue, 5340",Ireland,2023-05-15 02:39:02.003,Zara,413403,Fruit and vegetables,2023-09-16T10:55:08.803Z,Rodwell,6-777-367-783,Zara <zara_rodwell1398442854@nickia.com> Conta...,2021-01-09,Ireland,Ireland,zara_rodwell1398442854@nickia.com
2,"Abourne Lane, 876",Ireland,2023-05-15 02:39:02.003,Zara,417951,Milling,2023-09-16T10:53:14.079Z,Freeburn,5-618-556-540,Zara <zara_freeburn1593147546@gmail.com> Conta...,2021-08-30,Ireland,Ireland,zara_freeburn1593147546@gmail.com
3,"Chester Crossroad, 7070",Dublin,2023-05-15 02:39:02.003,Winnie,419852,Dairy products,2023-09-16T10:55:08.759Z,Walter,1-161-604-327,Winnie <winnie_walter538064895@sheye.org> Cont...,2021-02-10,Ireland,Dublin,winnie_walter538064895@sheye.org
4,"Tilloch Crossroad, 8332",Dublin,2023-05-15 02:39:02.003,Zoe,425352,Meat,2023-09-16T10:55:09.707Z,Owen,5-645-416-200,Zoe <zoe_owen1652446013@bungar.biz> Contact Info.,2021-11-02,Ireland,Dublin,zoe_owen1652446013@bungar.biz


## Fix phone number

In [16]:
data["contry_found"].unique()

array(['Ireland', 'United Kingdom'], dtype=object)

In [17]:
# Having only two countries it is more convenient to use a dictionary to normalize than to implement some kind of external library
dict_normalize_phones = {
    "Ireland":"(+353)",
    "United Kingdom": "(+44)"
}

In [18]:
data["fix_phone"] = data[["phone","contry_found"]].apply(lambda x: ETL.normalize_phones(
    dict_normalize_phones = dict_normalize_phones,
    phone = x.iloc[0], country = x.iloc[1]), axis = 1)

data.head(3)

,address,country,createdate,firstname,hs_object_id,industry,lastmodifieddate,lastname,phone,raw_email,technical_test___create_date,contry_found,city_found,email,fix_phone
0,"Blackpool Rue, 6576",Waterford,2023-05-15 02:39:02.002,Zoe,416102,Poultry and fish,2023-09-16T10:56:44.913Z,Owen,0-774-386-624,Zoe <zoe_owen450104633@acrit.org> Contact Info.,2021-07-13,Ireland,Waterford,zoe_owen450104633@acrit.org,(+353) 774386624
1,"Parkfield Avenue, 5340",Ireland,2023-05-15 02:39:02.003,Zara,413403,Fruit and vegetables,2023-09-16T10:55:08.803Z,Rodwell,6-777-367-783,Zara <zara_rodwell1398442854@nickia.com> Conta...,2021-01-09,Ireland,Ireland,zara_rodwell1398442854@nickia.com,(+353) 6777367783
2,"Abourne Lane, 876",Ireland,2023-05-15 02:39:02.003,Zara,417951,Milling,2023-09-16T10:53:14.079Z,Freeburn,5-618-556-540,Zara <zara_freeburn1593147546@gmail.com> Conta...,2021-08-30,Ireland,Ireland,zara_freeburn1593147546@gmail.com,(+353) 5618556540


## Conact name

In [19]:
data["names"] = data[["firstname","lastname"]].apply(lambda x: f"{x.iloc[0]} {x.iloc[1]}", axis = 1)
data.head(4)

,address,country,createdate,firstname,hs_object_id,industry,lastmodifieddate,lastname,phone,raw_email,technical_test___create_date,contry_found,city_found,email,fix_phone,names
0,"Blackpool Rue, 6576",Waterford,2023-05-15 02:39:02.002,Zoe,416102,Poultry and fish,2023-09-16T10:56:44.913Z,Owen,0-774-386-624,Zoe <zoe_owen450104633@acrit.org> Contact Info.,2021-07-13,Ireland,Waterford,zoe_owen450104633@acrit.org,(+353) 774386624,Zoe Owen
1,"Parkfield Avenue, 5340",Ireland,2023-05-15 02:39:02.003,Zara,413403,Fruit and vegetables,2023-09-16T10:55:08.803Z,Rodwell,6-777-367-783,Zara <zara_rodwell1398442854@nickia.com> Conta...,2021-01-09,Ireland,Ireland,zara_rodwell1398442854@nickia.com,(+353) 6777367783,Zara Rodwell
2,"Abourne Lane, 876",Ireland,2023-05-15 02:39:02.003,Zara,417951,Milling,2023-09-16T10:53:14.079Z,Freeburn,5-618-556-540,Zara <zara_freeburn1593147546@gmail.com> Conta...,2021-08-30,Ireland,Ireland,zara_freeburn1593147546@gmail.com,(+353) 5618556540,Zara Freeburn
3,"Chester Crossroad, 7070",Dublin,2023-05-15 02:39:02.003,Winnie,419852,Dairy products,2023-09-16T10:55:08.759Z,Walter,1-161-604-327,Winnie <winnie_walter538064895@sheye.org> Cont...,2021-02-10,Ireland,Dublin,winnie_walter538064895@sheye.org,(+353) 1161604327,Winnie Walter


# Duplicates Management

In [20]:
#Review empty data
data.isna().sum()

address                          0
country                          0
createdate                       0
firstname                       52
hs_object_id                     0
industry                         0
lastmodifieddate                 0
lastname                        52
phone                            0
raw_email                        0
technical_test___create_date     0
contry_found                     0
city_found                       0
email                            0
fix_phone                        0
names                            0
dtype: int64

## Review uniques email

In [21]:
temporal_vector = []
for email in tqdm(data["email"].unique(), desc="Extracting"):
    #print(email)
    data_tmp = data[data["email"] == email].copy()
    
    if len(data_tmp) != 1:
        # Find duplicate
        data_tmp = data_tmp.sort_values( by = "createdate", ascending = False).reset_index(drop = True)
        industry = ";".join(list(data_tmp["industry"].unique()))
        data_tmp.loc.__setitem__((0, ('industry')), industry)
        temporal_vector.append(data_tmp.head(1))
    else:
        temporal_vector.append(data_tmp)

Extracting:   0%|          | 0/165 [00:00<?, ?it/s]

In [22]:
data_postprocessing = pd.concat(temporal_vector)

In [23]:
data_postprocessing["industry"].unique()

array(['Poultry and fish', 'Fruit and vegetables', 'Milling',
       'Dairy products', 'Meat', 'Animal feeds', 'Bakery products',
       'Bakery products;Poultry and fish;Meat'], dtype=object)

In [24]:
#Review empty data
data_postprocessing.isna().sum()

address                         0
country                         0
createdate                      0
firstname                       1
hs_object_id                    0
industry                        0
lastmodifieddate                0
lastname                        1
phone                           0
raw_email                       0
technical_test___create_date    0
contry_found                    0
city_found                      0
email                           0
fix_phone                       0
names                           0
dtype: int64

In [25]:
# Others empty
data_postprocessing[data_postprocessing["firstname"].isna()] 

,address,country,createdate,firstname,hs_object_id,industry,lastmodifieddate,lastname,phone,raw_email,technical_test___create_date,contry_found,city_found,email,fix_phone,names
0,"Chester Tunnel, 3543",Limerick,2023-05-15 02:39:02.021,NaN,424102,Bakery products;Poultry and fish;Meat,2023-09-16T10:57:48.315Z,NaN,6-835-027-381,Tyson <tyson_newman1021718471@yahoo.com> Conta...,2021-09-21,Ireland,Limerick,tyson_newman1021718471@yahoo.com,(+353) 6835027381,nan nan


In [26]:
# This record is deleted because we do not have complete information, although it can be assumed that his name is tyson and his last name is newman and his email address, it is decided not to display any information
data_postprocessing = data_postprocessing[~data_postprocessing["firstname"].isna()].reset_index(drop=True)

In [28]:
data_postprocessing.to_csv("data/data_processing.csv", sep="|",index=False)

# Saving data